In [171]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import time  
from fake_useragent import UserAgent

from datetime import timedelta, date
import random


In [150]:
# Буду парсить сайт vesti.ru
# Данные получились не богатыми, но их можно использовать, например, для поиска трендов и актуальных событий


In [174]:
def get_page_links(page_date):
    """
        На вход подается дата в формате dd.mm.yyyy
        Возвращает список ссылок на новости, полученный с текущей страницы
    """
    page_link = 'https://www.vesti.ru/section.html?cid=6&date={}'.format(page_date)
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        return []
    
    try:
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        obj = soup.findAll('h3', attrs = {'class': 'b-item__title'})
        itog = [i.a.get('href') for i in obj]
    except:
        itog = None
        
    return itog


In [90]:
get_page_links('19.03.2020')


['/doc.html?id=3249513',
 '/doc.html?id=3249503',
 '/doc.html?id=3249499',
 '/doc.html?id=3249480',
 '/doc.html?id=3249481',
 '/doc.html?id=3249496',
 '/doc.html?id=3249485',
 '/doc.html?id=3249470',
 '/doc.html?id=3249463',
 '/doc.html?id=3249461',
 '/doc.html?id=3249401',
 '/doc.html?id=3249449',
 '/doc.html?id=3249459',
 '/doc.html?id=3249451',
 '/doc.html?id=3249447',
 '/doc.html?id=3249435',
 '/doc.html?id=3249422',
 '/doc.html?id=3249376',
 '/doc.html?id=3249426',
 '/doc.html?id=3249421',
 '/doc.html?id=3249348',
 '/doc.html?id=3249399',
 '/doc.html?id=3249403',
 '/doc.html?id=3249394',
 '/doc.html?id=3249393',
 '/doc.html?id=3249377',
 '/doc.html?id=3249378',
 '/doc.html?id=3249337',
 '/doc.html?id=3249287',
 '/doc.html?id=3249359',
 '/doc.html?id=3249354',
 '/doc.html?id=3249360',
 '/doc.html?id=3249326',
 '/doc.html?id=3249301',
 '/doc.html?id=3249342',
 '/doc.html?id=3249320',
 '/doc.html?id=3249298',
 '/doc.html?id=3249316',
 '/doc.html?id=3249312',
 '/doc.html?id=3249282',


In [278]:
def get_data(url_page):
    """
        Возвращает обработанный словарь с данными по одной новости
    """
    
    # запрашиваем данные по ссылке
    try:
        page_link = url_page
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    except:
        return None
    
    if not response.ok:
        return None
    
    # получаем содержимое страницы и переводим в суп
    try:
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
    except:
        return None
    
    title = None
    description = None
    keywords_text = None
    
    # Получаем основные теги:
    try:
        for tag in soup.head.find_all("meta"):
            if tag.get("name", None) == "keywords":
                keywords_text = tag.get("content", None)
                if keywords_text:
                    keywords_text = keywords_text.split(' ')
            elif tag.get("property", None) == "og:title":
                title = tag.get("content", None)
                
            elif tag.get("property", None) == "og:description":
                description = tag.get("content", None)
    except:
        title = None
        description = None
        keywords_text = None
        
    # Получаем время публикации
    try:
        obj_date = soup.find('div', attrs = {'class': 'article__date'})
        date_post = obj_date.text
        date_post = " ".join(date_post.split())
        time_post = date_post[:-5] + ' ' + date_post[-5:]
        time_post
    except:
        time_post = None
        
    # Получаем текст самой новости:
    try:
        obj_text = soup.find('div', attrs = {'class': 'article__text'})
        base_text = obj_text.text.split('!function')[0]
        base_text = " ".join(base_text.split())
        base_text
    except:
        base_text = None
    
    # составляем словарь, в котором будут хранится все полученные и обработанные данные

    data_row = {'title': title,
                'description': description,
                'keywords_text': keywords_text,
                'time_post': time_post,
                'base_text': base_text,
               }

    return data_row


In [160]:
get_data('https://www.vesti.ru/doc.html?id=3250104')


{'title': 'Минфин пообещал дополнительную поддержку отраслям экономики',
 'description': 'Правительству нужно будет оказать дополнительную поддержку отраслям экономики, пострадавшим от форс-мажорной ситуации на рынке. Об этом министр финансов Антон Силуанов заявил в программе "Москва. Кремль. Путин" на телеканале "Россия 1".',
 'keywords': ['силуанов', 'орешкин', 'экономика', 'поддержка'],
 'time_post': '22 марта 2020 13:41',
 'base_text': 'Правительству нужно будет оказать дополнительную поддержку отраслям экономики, пострадавшим от форс-мажорной ситуации на рынке. Об этом министр финансов Антон Силуанов заявил в программе "Москва. Кремль. Путин" на телеканале "Россия 1". "Это, безусловно, будет серьезное дополнительное испытание для российской экономики. С другой стороны, для бюджета созданы резервы. Уверены в том, что все расходы, которые были запланированы, профинансируем. Нам, конечно, нужно будет оказывать дополнительную поддержку отраслям экономики", — подчеркнул Силуанов. В сво

In [199]:
# функция-итерация по датам по одному дню:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


In [200]:
# Соберем все ссылки на новости

def get_list_links(start_date, end_date):
    page_links = []
    for single_date in daterange(start_date, end_date):
        mydata = single_date.strftime("%d.%m.%Y")
        page_links = page_links + get_page_links(mydata)
        pause = random.uniform(0.4, 1.)
        time.sleep(pause)

    return page_links


In [202]:
%%time
start_date = date(2020, 2, 1)
end_date = date(2020, 2, 29)

list_links = get_list_links(start_date, end_date)

len(list_links)


987

In [270]:
final_data = pd.DataFrame(columns=['title', 'description', 'keywords_text', 'time_post', 'base_text'])
final_data


,title,description,keywords,time_post,base_text


In [279]:
%%time
for link in list_links:
    link = 'https://www.vesti.ru' + link
    new_news = get_data(link)
    if new_news:
        final_data = final_data.append(new_news, ignore_index = True)
    pause = random.uniform(0.5, 1.)
    time.sleep(pause)


CPU times: user 2min 20s, sys: 2.93 s, total: 2min 23s
Wall time: 19min 19s


In [283]:
final_df = final_data.copy()
final_df = final_df.drop('keywords', axis = 1)


In [288]:
final_df.head()


,title,description,time_post,base_text,keywords_text
0,Пушков прокомментировал заявление Помпео о пос...,Российский сенатор Алексей Пушков оценил заявл...,1 февраля 2020 23:41,Российский сенатор Алексей Пушков оценил заявл...,"[пайетт, нефть, россия, белоруссия, пушков, се..."
1,Затулин рассказал о влюбленности Лукашенко,"""Это манипулирование"": так охарактеризовал пер...",1 февраля 2020 17:36,"""Это манипулирование"": так охарактеризовал пер...","[сша, госдепартамент, лукашенко, помпео, белор..."
2,"США готовы обеспечить Белоруссию нефтью, но са...",Америка как самый крупный энергопроизводитель ...,1 февраля 2020 14:22,Америка как самый крупный энергопроизводитель ...,"[помпео, лукашенко, посол, нефть, энергоресурс..."
3,"Рабинович пригрозил свержением Зеленскому и ""с...",Продолжение курса на узаконивание продажи земл...,1 февраля 2020 12:27,Продолжение курса на узаконивание продажи земл...,"[украина, земля, продажа, рабинович, за, жизнь..."
4,Транспорт vs экология в Иране: денег на бензин...,После осеннего трехкратного повышения цен на б...,1 февраля 2020 12:10,"Москва, 1 февраля - ""Вести.Экономика"". Иран не...",None


In [289]:
final_df.shape


(1311, 5)

In [292]:
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 5 columns):
title            1311 non-null object
description      1311 non-null object
time_post        1311 non-null object
base_text        1311 non-null object
keywords_text    1181 non-null object
dtypes: object(5)
memory usage: 51.3+ KB
